In [1]:
# Importing libraries
import nltk
import numpy as np
import pandas as pd
import random
from sklearn.model_selection import train_test_split
import pprint, time
 
#download the treebank corpus from nltk
nltk.download('treebank')
 
#download the universal tagset from nltk
nltk.download('universal_tagset')
 
# reading the Treebank tagged sentences
nltk_data = list(nltk.corpus.treebank.tagged_sents(tagset='universal'))
 
#print the first two sentences along with tags
print(nltk_data[:2])

[nltk_data] Downloading package treebank to /root/nltk_data...
[nltk_data]   Unzipping corpora/treebank.zip.
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.
[[('Pierre', 'NOUN'), ('Vinken', 'NOUN'), (',', '.'), ('61', 'NUM'), ('years', 'NOUN'), ('old', 'ADJ'), (',', '.'), ('will', 'VERB'), ('join', 'VERB'), ('the', 'DET'), ('board', 'NOUN'), ('as', 'ADP'), ('a', 'DET'), ('nonexecutive', 'ADJ'), ('director', 'NOUN'), ('Nov.', 'NOUN'), ('29', 'NUM'), ('.', '.')], [('Mr.', 'NOUN'), ('Vinken', 'NOUN'), ('is', 'VERB'), ('chairman', 'NOUN'), ('of', 'ADP'), ('Elsevier', 'NOUN'), ('N.V.', 'NOUN'), (',', '.'), ('the', 'DET'), ('Dutch', 'NOUN'), ('publishing', 'VERB'), ('group', 'NOUN'), ('.', '.')]]


In [2]:
#print each word with its respective tag for first two sentences
for sent in nltk_data[:1]:
  for tuple in sent:
    print(tuple)

('Pierre', 'NOUN')
('Vinken', 'NOUN')
(',', '.')
('61', 'NUM')
('years', 'NOUN')
('old', 'ADJ')
(',', '.')
('will', 'VERB')
('join', 'VERB')
('the', 'DET')
('board', 'NOUN')
('as', 'ADP')
('a', 'DET')
('nonexecutive', 'ADJ')
('director', 'NOUN')
('Nov.', 'NOUN')
('29', 'NUM')
('.', '.')


In [3]:
len(nltk_data)

3914

In [4]:
# split data into training and validation set in the ratio 80:20
train_set,test_set =train_test_split(nltk_data,train_size=0.80,test_size=0.20,random_state = 101)

In [5]:
# create list of train and test tagged words
train_tagged_words = [ tup for sent in train_set for tup in sent ]
test_tagged_words = [ tup for sent in test_set for tup in sent ]
print(len(train_tagged_words))
print(len(test_tagged_words))

80310
20366


In [6]:
# check some of the tagged words.
train_tagged_words[:5]

[('Drink', 'NOUN'),
 ('Carrier', 'NOUN'),
 ('Competes', 'VERB'),
 ('With', 'ADP'),
 ('Cartons', 'NOUN')]

In [7]:
#use set datatype to check how many unique tags are present in training data
tags = {tag for word,tag in train_tagged_words}
print(len(tags))
print(tags)
 
# check total words in vocabulary
vocab = {word for word,tag in train_tagged_words}

12
{'CONJ', '.', 'PRT', 'ADJ', 'ADP', 'NUM', 'X', 'DET', 'ADV', 'NOUN', 'VERB', 'PRON'}


In [8]:
def word_given_tag1(word, tag, train_bag = train_tagged_words):
    tag_list = [pair for pair in train_bag if pair[1]==tag]
    count_tag = len(tag_list)#total number of times the passed tag occurred in train_bag
    w_given_tag_list = [pair[0] for pair in tag_list if pair[0]==word]
#now calculate the total number of times the passed word occurred as the passed tag.
    count_w_given_tag = len(w_given_tag_list)
 
     
    return count_w_given_tag/ count_tag

In [9]:
# compute Emission Probability
memo={}
def word_given_tag(word, tag, train_bag = train_tagged_words):
    tag_dict=memo.get(word,-1)
    if tag_dict!=-1 and tag_dict.get(tag,-1) !=-1:
      return memo[word][tag]
    tag_list = [pair for pair in train_bag if pair[1]==tag]
    count_tag = len(tag_list)#total number of times the passed tag occurred in train_bag
    w_given_tag_list = [pair[0] for pair in tag_list if pair[0]==word]
#now calculate the total number of times the passed word occurred as the passed tag.
    count_w_given_tag = len(w_given_tag_list)
    emission_prop=count_w_given_tag/ count_tag
    if tag_dict ==-1: 
      memo[word]={tag:emission_prop}
    else :
      memo[word][tag]=emission_prop
 
     
    return  memo[word][tag]

In [10]:
memo

{}

In [11]:
# compute  Transition Probability
def t2_given_t1(t2, t1, train_bag = train_tagged_words):
    tags = [pair[1] for pair in train_bag]
    count_t1 = len([t for t in tags if t==t1])
    count_t2_t1 = 0
    for index in range(len(tags)-1):
        if tags[index]==t1 and tags[index+1] == t2:
            count_t2_t1 += 1
    return count_t2_t1/count_t1

In [12]:
# creating t x t transition matrix of tags, t= no of tags
# Matrix(i, j) represents P(jth tag after the ith tag)
 
tags_matrix = np.zeros((len(tags), len(tags)), dtype='float32')
for i, t1 in enumerate(list(tags)):
    for j, t2 in enumerate(list(tags)): 
        tags_matrix[i, j] = t2_given_t1(t2, t1)
 
print(tags_matrix)

[[5.48847427e-04 3.51262353e-02 4.39077942e-03 1.13611415e-01
  5.59824370e-02 4.06147093e-02 9.33040585e-03 1.23490669e-01
  5.70801310e-02 3.49066973e-01 1.50384188e-01 6.03732169e-02]
 [6.00793920e-02 9.23720598e-02 2.78940029e-03 4.61323895e-02
  9.29084867e-02 7.82104954e-02 2.56410260e-02 1.72191828e-01
  5.25694676e-02 2.18538776e-01 8.96899477e-02 6.87694475e-02]
 [2.34833662e-03 4.50097844e-02 1.17416831e-03 8.29745606e-02
  1.95694715e-02 5.67514673e-02 1.21330721e-02 1.01369865e-01
  9.39334650e-03 2.50489235e-01 4.01174158e-01 1.76125243e-02]
 [1.68932043e-02 6.60194159e-02 1.14563107e-02 6.33009672e-02
  8.05825219e-02 2.17475723e-02 2.09708735e-02 5.24271838e-03
  5.24271838e-03 6.96893215e-01 1.14563107e-02 1.94174761e-04]
 [1.01240189e-03 3.87243740e-02 1.26550242e-03 1.07061505e-01
  1.69577319e-02 6.32751212e-02 3.45482156e-02 3.20931405e-01
  1.45532778e-02 3.23588967e-01 8.47886596e-03 6.96026310e-02]
 [1.42806144e-02 1.19243130e-01 2.60621198e-02 3.53445187e-02
  3

In [13]:
len(tags)

12

In [14]:
# convert the matrix to a df for better readability
#the table is same as the transition table shown in section 3 of article
tags_df = pd.DataFrame(tags_matrix, columns = list(tags), index=list(tags))
display(tags_df)

,CONJ,.,PRT,ADJ,ADP,NUM,X,DET,ADV,NOUN,VERB,PRON
CONJ,0.000549,0.035126,0.004391,0.113611,0.055982,0.040615,0.009330,0.123491,0.057080,0.349067,0.150384,0.060373
.,0.060079,0.092372,0.002789,0.046132,0.092908,0.078210,0.025641,0.172192,0.052569,0.218539,0.089690,0.068769
PRT,0.002348,0.045010,0.001174,0.082975,0.019569,0.056751,0.012133,0.101370,0.009393,0.250489,0.401174,0.017613
ADJ,0.016893,0.066019,0.011456,0.063301,0.080583,0.021748,0.020971,0.005243,0.005243,0.696893,0.011456,0.000194
ADP,0.001012,0.038724,0.001266,0.107062,0.016958,0.063275,0.034548,0.320931,0.014553,0.323589,0.008479,0.069603
NUM,0.014281,0.119243,0.026062,0.035345,0.037487,0.184220,0.202428,0.003570,0.003570,0.351660,0.020707,0.001428
X,0.010379,0.160869,0.185086,0.017682,0.142226,0.003075,0.075726,0.056890,0.025754,0.061695,0.206419,0.054200
DET,0.000431,0.017393,0.000287,0.206411,0.009918,0.022855,0.045134,0.006037,0.012074,0.635906,0.040247,0.003306
ADV,0.006982,0.139255,0.014740,0.130721,0.119472,0.029868,0.022886,0.071373,0.081458,0.032196,0.339022,0.012025
NOUN,0.042454,0.240094,0.043935,0.012584,0.176827,0.009144,0.028825,0.013106,0.016895,0.262344,0.149134,0.004659


In [15]:
def Viterbi(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
     
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['X', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                 
            # compute emission and state probabilities
            emission_p = word_given_tag(words[key], tag)
            state_probability =emission_p* transition_p    
            p.append(state_probability)
        pmax = max(p)
        # getting state for which probability is maximum
        state_max = T[p.index(pmax)] 
        state.append(state_max)
    return list(zip(words, state))

O(#Words * #Tags  * len(train_bag))

In [16]:
# Let's test our Viterbi algorithm on a few sample sentences of test dataset
random.seed(1234)      #define a random seed to get same sentences when run multiple times
 
# choose random 10 numbers
rndom = [random.randint(1,len(test_set)-1) for _ in range(100)]
# list of 10 sents on which we test the model
test_run = [test_set[i] for i in rndom]
 
# list of tagged words
test_run_base = [tup for sent in test_run for tup in sent]
 
# list of untagged words
test_tagged_words = [tup[0] for sent in test_run for tup in sent]
print(len(test_tagged_words))

2557


In [17]:
#Here We will only test 10 sentences to check the accuracy
#as testing the whole training set takes huge amount of time
start = time.time()
tagged_seq = Viterbi(test_tagged_words)
end = time.time()
difference = end-start
 
print("Time taken in seconds: ", difference)
 
# accuracy
check = [i for i, j in zip(tagged_seq, test_run_base) if i == j] 
 
accuracy = len(check)/len(tagged_seq)
print('Viterbi Algorithm Accuracy: ',accuracy*100)

Time taken in seconds:  92.17855429649353
Viterbi Algorithm Accuracy:  91.00508408290966


In [18]:
sent='All my sins need holy water, feel it washing over me .'
words=sent.split()
# words=[ tup[0] for tup in test_set[0]]
print(sent)
start = time.time()
tagged_seq = Viterbi(words)
end = time.time()
difference = end-start
 
print("Time taken in seconds: ", difference)
print(tagged_seq)

All my sins need holy water, feel it washing over me .
Time taken in seconds:  0.35845255851745605
[('All', 'DET'), ('my', 'PRON'), ('sins', 'CONJ'), ('need', 'VERB'), ('holy', 'CONJ'), ('water,', 'CONJ'), ('feel', 'VERB'), ('it', 'PRON'), ('washing', 'CONJ'), ('over', 'ADP'), ('me', 'PRON'), ('.', '.')]


In [ ]:
len(test_tagged_words)

2557

In [ ]:
print(len(test_set))

783


In [ ]:
#Code to test all the test sentences
#(takes alot of time to run s0 we wont run it here)
# tagging the test sentences()
test_tagged_words = [tup for sent in test_set for tup in sent]
test_untagged_words = [tup[0] for sent in test_set for tup in sent]
test_untagged_words
print(len(test_untagged_words))
start = time.time()
tagged_seq = Viterbi(test_untagged_words)
end = time.time()
difference = end-start
 
print("Time taken in seconds: ", difference)
 
# accuracy
check = [i for i, j in zip(test_tagged_words, test_untagged_words) if i == j] 
 
accuracy = len(check)/len(tagged_seq)
print('Viterbi Algorithm Accuracy: ',accuracy*100)

20366


In [19]:
#To improve the performance,we specify a rule base tagger for unknown words 
# specify patterns for tagging
patterns = [
    (r'.*ing$', 'VERB'),              # gerund
    (r'.*ed$', 'VERB'),               # past tense 
    (r'.*es$', 'VERB'),               # verb    
    (r'.*\'s$', 'NOUN'),              # possessive nouns
    (r'.*s$', 'NOUN'),                # plural nouns
    (r'\*T?\*?-[0-9]+$', 'X'),        # X
    (r'^-?[0-9]+(.[0-9]+)?$', 'NUM'), # cardinal numbers
    (r'.*', 'NOUN')                   # nouns
]
 
# rule based tagger
rule_based_tagger = nltk.RegexpTagger(patterns)

In [26]:
#modified Viterbi to include rule based tagger in it
def Viterbi_rule_based(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
     
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                 
            # compute emission and state probabilities
            emission_p = word_given_tag(words[key], tag)
            state_probability = emission_p * transition_p    
            p.append(state_probability)
             
        pmax = max(p)
        state_max = rule_based_tagger.tag([word])[0][1]       
        
         
        if(pmax==0):
            state_max = rule_based_tagger.tag([word])[0][1] # assign based on rule based tagger
        else:
            if state_max != 'X':
                # getting state for which probability is maximum
                state_max = T[p.index(pmax)]                
             
         
        state.append(state_max)
    return list(zip(words, state))

In [27]:
#test accuracy on subset of test data 
start = time.time()
tagged_seq = Viterbi_rule_based(test_tagged_words)
end = time.time()
difference = end-start
 
print("Time taken in seconds: ", difference)
 
# accuracy
check = [i for i, j in zip(tagged_seq, test_run_base) if i == j] 
 
accuracy = len(check)/len(tagged_seq)
print('Viterbi Algorithm Accuracy: ',accuracy*100)

Time taken in seconds:  0.4493680000305176
Viterbi Algorithm Accuracy:  95.30700039108329


In [28]:
#Check how a sentence is tagged by the two POS taggers
#and compare them
test_sent="Will can see Marry"
pred_tags_rule=Viterbi_rule_based(test_sent.split())
pred_tags_withoutRules= Viterbi(test_sent.split())
print(pred_tags_rule)
print(pred_tags_withoutRules)
#Will and Marry are tagged as NUM as they are unknown words for Viterbi Algorithm 

[('Will', 'NOUN'), ('can', 'VERB'), ('see', 'VERB'), ('Marry', 'NOUN')]
[('Will', 'CONJ'), ('can', 'VERB'), ('see', 'VERB'), ('Marry', 'CONJ')]
